In [1]:
import numpy as np
import os
import sys
sys.path.append('../')
import cv2

In [2]:
from utils import load_datasets

In [14]:
def load_datasets(datasets_dir, multi_class=True, read_from_stubs=False, save_stub=False):
    """Given a directory containing all our datasets, return our train and validation data in 2 numpy array.

    Args:
        datasets_dir (str): path/to/dataset_dir
        multi_class (bool, optional): Choose if you do multi-class or binary classification. Defaults to True.
        read_from_stubs (bool, optional): Read data from npy file (faster). Defaults to False.
        save_stub (bool, optional): Save a npy file. Defaults to False.

    Returns:
        np.array: train and validation datasets in np.array
    """
    if read_from_stubs and (os.path.exists(os.path.join(datasets_dir, 'dataset_01.npy')) and os.path.exists(os.path.join(datasets_dir, 'dataset_02.npy'))):
        with open(os.path.join(datasets_dir, 'dataset_01.npy'), 'rb') as f:
            dataset_01 = np.load(f)
        with open(os.path.join(datasets_dir, 'dataset_02.npy'), 'rb') as f:
            dataset_02 = np.load(f)
        
        return dataset_01, dataset_02
            
            
    train_total_size = 0
    val_total_size = 0
    
    if multi_class:
        for folder in os.listdir(datasets_dir):
            if folder.endswith('01'):
                train_total_size  += len(os.listdir(os.path.join(datasets_dir, folder)))
            elif folder.endswith('02'):
                val_total_size += len(os.listdir(os.path.join(datasets_dir, folder)))  
    
    else:
        for folder in os.listdir(datasets_dir)[:4]:
            if folder.endswith('01'):
                train_total_size  += len(os.listdir(os.path.join(datasets_dir, folder)))
            elif folder.endswith('02'):
                val_total_size += len(os.listdir(os.path.join(datasets_dir, folder)))
        
    dataset_01 = np.empty(shape=(train_total_size, 540, 920, 3), dtype=np.float32)
    dataset_02 = np.empty(shape=(val_total_size, 540, 920, 3), dtype=np.float32)
        
    i = 0
    for folder in os.listdir(datasets_dir):
        if folder.endswith('01'):
            for file in os.listdir(os.path.join(datasets_dir, folder)):
                dataset_01[i] = cv2.imread(os.path.join(datasets_dir, folder, file)) / 255.0
                i += 1
    i = 0
    for folder in os.listdir(datasets_dir):
        if folder.endswith('02'):
            for file in os.listdir(os.path.join(datasets_dir, folder)):
                dataset_02[i] = cv2.imread(os.path.join(datasets_dir, folder, file)) / 255.0
                i += 1
    
    if save_stub:
        with open(os.path.join(datasets_dir, 'dataset_01.npy'), 'wb') as f:
            np.save(f, dataset_01)
        with open(os.path.join(datasets_dir, 'dataset_02.npy'), 'wb') as f:
            np.save(f, dataset_02)

    return dataset_01, dataset_02

In [55]:
datasets_dir = '../datasets/Movies'
multi_class = False

In [74]:
datasets_dir = '../datasets/Movies'
train_total_size = 0
val_total_size = 0
multi_class = True

if multi_class:
        for folder in os.listdir(datasets_dir):
            if folder.endswith('01'):
                train_total_size  += len(os.listdir(os.path.join(datasets_dir, folder)))
            elif folder.endswith('02'):
                val_total_size += len(os.listdir(os.path.join(datasets_dir, folder)))      
else:
    for folder in os.listdir(datasets_dir)[:4]:
        if folder.endswith('01'):
            train_total_size  += len(os.listdir(os.path.join(datasets_dir, folder)))
        elif folder.endswith('02'):
            val_total_size += len(os.listdir(os.path.join(datasets_dir, folder)))

In [75]:
print(int(0.8*train_total_size))
print(int(0.2*train_total_size))

1600
400


In [76]:
dataset_01_train = np.empty(shape=(int(0.8*train_total_size), 540, 920, 3), dtype=np.float32)
dataset_01_test = np.empty(shape=(int(0.2*train_total_size), 540, 920, 3), dtype=np.float32)
dataset_02 = np.empty(shape=(val_total_size, 540, 920, 3), dtype=np.float32)

In [77]:
idx_test = 0
idx_train = 0
idx_val = 0
for folder in os.listdir(datasets_dir):
    len_folder = len(os.listdir(os.path.join(datasets_dir, folder)))  
    if folder.endswith('01'):
        for i, file in enumerate(os.listdir(os.path.join(datasets_dir, folder))[:int(0.8*len_folder)]):
            dataset_01_train[i+idx_train] = cv2.imread(os.path.join(datasets_dir, folder, file)) / 255.0
        idx_train += int(0.8*len_folder)
        for i, file in enumerate(os.listdir(os.path.join(datasets_dir, folder))[int(0.8*len_folder):]):
            dataset_01_test[i+idx_test] = cv2.imread(os.path.join(datasets_dir, folder, file)) / 255.0
        idx_test += int(0.2*len_folder)
    if folder.endswith('02'):
        for file in os.listdir(os.path.join(datasets_dir, folder)):
            dataset_02[idx_val] = cv2.imread(os.path.join(datasets_dir, folder, file)) / 255.0
            idx_val += 1
        


In [84]:
with open(os.path.join(datasets_dir, 'ds01_train.npy'), 'wb') as f:
    np.save(f, dataset_01_train)
with open(os.path.join(datasets_dir, 'ds01_test.npy'), 'wb') as f:
    np.save(f, dataset_01_test)
with open(os.path.join(datasets_dir, 'ds02.npy'), 'wb') as f:
    np.save(f, dataset_02)


In [79]:
cv2.imshow('', dataset_02[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [56]:
print(len(dataset_01_train))

1600


In [3]:
datasets_dir = '../datasets/Movies'

In [6]:
with open(os.path.join(datasets_dir, 'ds01_train.npy'), 'rb') as f:
    dataset_01_train = np.load(f)



EOFError: No data left in file